In [51]:
%load_ext autoreload
%autoreload 2

from toolbox.data_prep_helpers import *
from toolbox.evaluation import *
import pandas as pd
import numpy as np

from models.bagofwords_classifier import create_model

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input, Flatten, Activation, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from keras.preprocessing.text import text
from sklearn.preprocessing import MultiLabelBinarizer


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
sample_size = 100000
n_top_labels = 100
normalize_embeddings = False
learning_rate = 0.0000001
vocab_size = 1000

## Load Data

In [12]:
data_path = "../data/pythonquestions/"
total_data = load_data(data_path)

loading data from cached pickle


In [13]:
data = total_data.sample(sample_size)

## Prepare Data

In [14]:
print(data.shape)
remove_html_tags(data, ["Body_q"])
data = data[data["tags"].apply(lambda tags: all([isinstance(t, str) for t in tags]))]
print(data.shape)
data = reduce_number_of_tags(data, 100)

(100000, 5)
(99915, 5)
deleting element python from top_tags


In [15]:
train_size = int(len(data) * .8)
train_posts = data['Body_q'][:train_size]
train_tags = data['tags'][:train_size]
test_posts = data['Body_q'][train_size:]
test_tags = data['tags'][train_size:]

In [16]:
tokenize = text.Tokenizer(num_words=vocab_size)
tokenize.fit_on_texts(train_posts)

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

In [17]:
encoder = MultiLabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)
n_col = y_train.shape[1]

## Create and Train Model

In [25]:
# Returned Model not Compiled yet to better play with optimizers!!
model = create_model(input_layer_size=256, vocab_size=vocab_size,output_dim=n_col)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 256)               256256    
_________________________________________________________________
activation_6 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               25700     
_________________________________________________________________
activation_7 (Activation)    (None, 100)               0         
Total params: 281,956
Trainable params: 281,956
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Optimizer
opt = SGD(lr=learning_rate, momentum=0.9)
opt_Adam = Adam(lr = learning_rate)

In [27]:
model.compile(loss='binary_crossentropy', optimizer='adam' , metrics=["accuracy"])

In [61]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.7))
model.add(Dense(n_col))
model.add(Activation('sigmoid'))

opt_Adam = Adam(lr = learning_rate)
model.compile(loss='binary_crossentropy', optimizer='adam' , metrics=["accuracy"])


In [62]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=20,
                    verbose=1,
                    validation_split=0.1)

Train on 50401 samples, validate on 5601 samples
Epoch 1/20


50401/50401 [==============================] - 3s 58us/sample - loss: 0.0781 - accuracy: 0.9814 - val_loss: 0.0425 - val_accuracy: 0.9884
Epoch 2/20
50401/50401 [==============================] - 3s 52us/sample - loss: 0.0415 - accuracy: 0.9887 - val_loss: 0.0355 - val_accuracy: 0.9892
Epoch 3/20
50401/50401 [==============================] - 2s 49us/sample - loss: 0.0364 - accuracy: 0.9893 - val_loss: 0.0336 - val_accuracy: 0.9895
Epoch 4/20
50401/50401 [==============================] - 2s 49us/sample - loss: 0.0340 - accuracy: 0.9897 - val_loss: 0.0326 - val_accuracy: 0.9897
Epoch 5/20
50401/50401 [==============================] - 3s 50us/sample - loss: 0.0326 - accuracy: 0.9900 - val_loss: 0.0323 - val_accuracy: 0.9898
Epoch 6/20
50401/50401 [==============================] - 3s 52us/sample - loss: 0.0315 - accuracy: 0.9902 - val_loss: 0.0321 - val_accuracy: 0.9899
Epoch 7/20
50401/50401 [==============================] - 3s 53us/sample - loss: 0.0307 - accuracy: 0.9903 - val_loss

In [45]:
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=20,
                    verbose=1,
                    validation_split=0.1)

Train on 50401 samples, validate on 5601 samples
Epoch 1/20
50401/50401 [==============================] - 3s 58us/sample - loss: 0.0583 - accuracy: 0.9856 - val_loss: 0.0360 - val_accuracy: 0.9893
Epoch 2/20
50401/50401 [==============================] - 3s 51us/sample - loss: 0.0320 - accuracy: 0.9901 - val_loss: 0.0331 - val_accuracy: 0.9898
Epoch 3/20
50401/50401 [==============================] - 2s 47us/sample - loss: 0.0286 - accuracy: 0.9907 - val_loss: 0.0328 - val_accuracy: 0.9897
Epoch 4/20
50401/50401 [==============================] - 2s 47us/sample - loss: 0.0265 - accuracy: 0.9912 - val_loss: 0.0329 - val_accuracy: 0.9897
Epoch 5/20
50401/50401 [==============================] - 2s 49us/sample - loss: 0.0247 - accuracy: 0.9917 - val_loss: 0.0334 - val_accuracy: 0.9897
Epoch 6/20
50401/50401 [==============================] - 3s 51us/sample - loss: 0.0231 - accuracy: 0.9922 - val_loss: 0.0344 - val_accuracy: 0.9896
Epoch 7/20
50401/50401 [==============================] -

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=opt , metrics=["accuracy"])

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)

## Evaluate Model

In [31]:
n_predictions = 300

predictions = model.predict(x_test[:n_predictions])

l_pred = encoder.inverse_transform(binarize_model_output(predictions, threshold=0.1))
l_true = encoder.inverse_transform(y_test[:n_predictions])
raw_texts = test_posts[:n_predictions]

for pred, act, txt, i in zip(l_pred, l_true, raw_texts, range(5)):
    print(f"TRUE: {act}\nPREDICTION: {pred}\n")
    print(txt)

TRUE: ('django',)
PREDICTION: ('datetime', 'django')

a= {'Locator__creation_date': {'start': datetime.date(2013, 11, 14), 'end': datetime.date(2013, 11, 14)}, 'Locator__employed': u'True', 'Locator__employer_state': u'AL', 'Locator__receiving_hiv_treatment': u'True', 'Locator__hiv_treatment_state': u'AR', 'Locator__mental_health_provider': u'False', 'Locator__parole': u'True', 'Locator__parole_state': u'IA', 'data_model_name': ['Locator']}

ast.literal_eval(a)

it gives
ValueError: malformed string


TRUE: ('scipy',)
PREDICTION: ('nltk', 'numpy', 'python-2.7', 'scipy', 'ubuntu')

I'm starting to work with Python and I've been trying to use some methods of SciPy and SciKits, but when I import the SciPy module I get this error:
ImportError: /usr/local/lib/python2.7/dist-packages/scipy/linalg/clapack.so: 
   undefined symbol: clapack_sgesv 

I've been looking for a solution but haven't found one yet. This is on Ubuntu 12.10. Can anyone help me?

TRUE: ('list',)
PREDICTION: ('algorithm', 

In [71]:
predictions = model.predict(x_test[:n_predictions])
# l_pred_binary = binarize_model_output(predictions, 0.27)
# l_true_binary = y_test[:n_predictions]
output_evaluation(model, sample_size, None, n_top_labels, l_true_binary, predictions, normalize_embeddings, learning_rate, vocab_size, n_epochs=20)

Model Evaluation

normalize_embeddings = False, learning_rate = 1e-07, vocab_size = 1000, epochs=20
Parameter Settings:
 Sample size = 100000, Max. number of words per question = None, Number of Top Labels used = 100

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 512)               512512    
_________________________________________________________________
activation_19 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 100)               51300     
_________________________________________________________________
activation_20 (Activation)   (None, 100)               0         
Total params: 563,812
Trainable p